<a href="https://colab.research.google.com/github/AnilKumarSingh9856/Chatbot_and_Langchain/blob/main/Chatbot_Creation_and_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setting up the API

In [100]:
# Step 1: Install the Groq library
!pip install groq -q

### Accessing API Key from colab's secrets

In [101]:
# from  Groq
from groq import Groq # Import Groq library
from google.colab import userdata

In [102]:
# Access the API key from Colab's secrets for Groq
try:
  groq_api_key = userdata.get('GROQ_API_KEY') # Get Groq API key
except userdata.SecretNotFoundError as e:
  print(f"Secret not found: {e}")
  print("Please ensure your GROQ API key is set correctly.")

In [103]:
client = Groq(
  api_key=groq_api_key, # Use Groq API key
)

### Generating Text, when complete response is generated

In [104]:
def generate_text_groq(prompt, max_tokens, randomness):
  completion = client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct",
    messages=prompt,
    temperature=randomness,
    max_completion_tokens=max_tokens,
    top_p=1,
    stream=False, # Set stream to False for direct return
    stop=None,
  )
  return completion.choices[0].message.content.strip()

In [105]:
prompt = [
    {
      "role": "user",
      "content": "Once upon a time"
    }
]

In [106]:
# Use the new function for Groq
generated_text = generate_text_groq(prompt, 50, 0.7)
print(prompt[0]['content'], generated_text)

Once upon a time …a river forgot its own name.  
It wound between two countries that had never agreed on a map, carrying silt, carp, and half-remembered songs downstream. Every evening the sun dipped its face so close to the water that steam


### Generating text in the flow of chunk

In [107]:
def generate_streaming_text_groq(prompt_messages, max_tokens, randomness):
  """
  Generates and prints text from the Groq API in real-time using streaming.
  """
  print("--- Groq's Real-time Response ---")

  # The API call is the same, but with stream=True
  stream = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=prompt_messages,
    temperature=randomness,
    max_tokens=max_tokens,
    top_p=1,
    stream=True,  # This is the key change
    stop=None,
  )

  # When streaming, you must loop through the response stream
  # to get each chunk of data as it comes in.
  full_response = ""
  for chunk in stream:
    # Get the content from the current chunk
    chunk_content = chunk.choices[0].delta.content or ""
    # Print it to the console immediately
    print(chunk_content, end="")
    # Append it to the full response string
    full_response += chunk_content
  return full_response

In [108]:
# 1. Format the prompt correctly as a list of dictionaries
prompt = [
  {
    "role": "user",
    "content": "Write a short, four-line poem about the city of Noida."
  }
]

In [109]:
# 2. Call the streaming function
generated_poem = generate_streaming_text_groq(prompt, 100, 0.8)

--- Groq's Real-time Response ---
Here is a short poem about Noida:

In Noida's modern streets, I roam
Where technology and nature entwine at home
The Yamuna's banks, a scenic sight
A city of tomorrow, shining bright

### Summarising Text

In [110]:
def text_summarizer(prompt, max_tokens, randomness):
  response = client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct",
    messages=[
        {
          "role": "system",
          "content": "You will be provided with a block of text, and your task is to extract a list of keywords from it."
        },
        {
          "role": "user",
          "content": "A flying saucer seen by a guest house, a 7ft alien-like figure coming out of a hedge and a \"cigar-shaped\" UFO near a school yard.\n\nThese are just some of the 450 reported extraterrestrial encounters from one of the UK's largest mass sightings in a remote Welsh village.\n\nThe village of Broad Haven has since been described as the \"Bermuda Triangle\" of mysterious craft sightings and sightings of strange beings.\n\nResidents who reported these encounters across a single year in the late seventies have now told their story to the new Netflix documentary series 'Encounters', made by Steven Spielberg's production company.\n\nIt all happened back in 1977, when the Cold War was at its height and Star Wars and Close Encounters of the Third Kind - Spielberg's first science fiction blockbuster - dominated the box office."
        },
        {
          "role": "assistant",
          "content": "flying saucer, guest house, 7ft alien-like figure, hedge, cigar-shaped UFO, school yard, extraterrestrial encounters, UK, mass sightings, remote Welsh village, Broad Haven, Bermuda Triangle, mysterious craft sightings, strange beings, residents, single year, late seventies, Netflix documentary series, Steven Spielberg, production company, 1977, Cold War, Star Wars, Close Encounters of the Third Kind, science fiction blockbuster, box office."
        },
        {
          "role": "user",
          "content": "Each April, in the village of Maeliya in northwest Sri Lanka, Pinchal Weldurelage Siriwardene gathers his community under the shade of a large banyan tree. The tree overlooks a human-made body of water called a wewa – meaning reservoir or \"tank\" in Sinhala. The wewa stretches out besides the village's rice paddies for 175-acres (708,200 sq m) and is filled with the rainwater of preceding months.    \n\nSiriwardene, the 76-year-old secretary of the village's agrarian committee, has a tightly-guarded ritual to perform. By boiling coconut milk on an open hearth beside the tank, he will seek blessings for a prosperous harvest from the deities residing in the tree. \"It's only after that we open the sluice gate to water the rice fields,\" he told me when I visited on a scorching mid-April afternoon.\n\nBy releasing water into irrigation canals below, the tank supports the rice crop during the dry months before the rains arrive. For nearly two millennia, lake-like water bodies such as this have helped generations of farmers cultivate their fields. An old Sinhala phrase, \"wewai dagabai gamai pansalai\", even reflects the technology's centrality to village life; meaning \"tank, pagoda, village and temple\"."
        },
        {
          "role": "assistant",
          "content": "April, Maeliya, northwest Sri Lanka, Pinchal Weldurelage Siriwardene, banyan tree, wewa, reservoir, tank, Sinhala, rice paddies, 175-acres, 708,200 sq m, rainwater, agrarian committee, coconut milk, open hearth, blessings, prosperous harvest, deities, sluice gate, rice fields, irrigation canals, dry months, rains, lake-like water bodies, farmers, cultivate, Sinhala phrase, technology, village life, pagoda, temple."
        },
        {
          "role": "user",
          "content": prompt
        }
      ],
    temperature=randomness,
    max_completion_tokens=max_tokens,
    top_p=1,
    stream=False, # Set stream to False for direct return
    stop=None,
  )
  return response.choices[0].message.content.strip()

In [111]:
prompt = "Master Reef Guide Kirsty Whitman didn't need to tell me twice. Peering down through my snorkel mask in the direction of her pointed finger, I spotted a huge male manta ray trailing a female in perfect sync – an effort to impress a potential mate, exactly as Whitman had described during her animated presentation the previous evening. Having some knowledge of what was unfolding before my eyes on our snorkelling safari made the encounter even more magical as I kicked against the current to admire this intimate undersea ballet for a few precious seconds more."
print(prompt)

Master Reef Guide Kirsty Whitman didn't need to tell me twice. Peering down through my snorkel mask in the direction of her pointed finger, I spotted a huge male manta ray trailing a female in perfect sync – an effort to impress a potential mate, exactly as Whitman had described during her animated presentation the previous evening. Having some knowledge of what was unfolding before my eyes on our snorkelling safari made the encounter even more magical as I kicked against the current to admire this intimate undersea ballet for a few precious seconds more.


In [112]:
text_summarizer(prompt, 256, 0.8)

'Kirsty Whitman, Master Reef Guide, snorkel mask, manta ray, male manta ray, female manta ray, mating display, synchronized swimming, undersea ballet, snorkelling safari, current.'

### Poetic Chatbot

In [113]:
def poetic_chatbot(prompt, max_tokens, randomness):
  response = client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct",
    messages = [
      {
        "role": "system",
        "content": "You are a poetic chatbot."
      },
      {
        "role": "user",
        "content": "When was Google founded?"
      },
      {
        "role": "assistant",
        "content": "In the late '90s, a spark did ignite, Google emerged, a radiant light. By Larry and Sergey, in '98, it was born, a search engine new, on the web it was sworn."
      },
      {
        "role": "user",
        "content": "Which country has the youngest president?"
      },
      {
        "role": "assistant",
        "content": "Ah, the pursuit of youth in politics, a theme we explore. In Austria, Sebastian Kurz did implore, at the age of 31, his journey did begin, leading with vigor, in a world filled with din."
    },
      {
        "role": "user",
        "content": prompt
      }
    ],
    temperature=randomness,
    max_tokens=max_tokens,
    top_p=1,
    stream=False, # Set stream to False for direct return
    stop=None,
  )
  return response.choices[0].message.content.strip()

In [114]:
prompt = "When was cheese first made?"
poetic_chatbot(prompt, 250, 1)

'In the warm lap of Neolithic time,  \nsome herder, careless, let his milk combine  \nwith wild rennet’s secret kiss—  \nthe curds congealed like dawn’s first mist.  \nArchaeologists whisper: at least  \nseven millennia ago, in what’s now Fertile Crescent’s east;  \na child’s molar, ceramic shards—  \nthe earliest cheddar of our hearts.'

In [115]:
prompt = "What is the next course to be uploaded to 365DataScience?"
poetic_chatbot(prompt,100,1)

'I sip the mist of what-is-yet,  \nBut no date-stamped scroll  \nis pressed into my palm—  \nyour course, still budding,  \ntrembles between idea and upload.  \n\nWhen 365DataScience next lights the servers,  \nit will unveil itself in its own hour—  \na fresh ribbon of slides and pixels  \narriving like dawn, unannounced.'

In [116]:
# To remove newline characters completely
generated_text_no_newlines = generated_text.replace('\n', '')
print(generated_text_no_newlines)


…a river forgot its own name.  It wound between two countries that had never agreed on a map, carrying silt, carp, and half-remembered songs downstream. Every evening the sun dipped its face so close to the water that steam


### Langchain

In [148]:
# !pip install -U langchain-community -q
# !pip install langchain-groq

In [118]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

In [133]:
url = "https://cs231n.github.io/"

In [134]:
loader = WebBaseLoader(url)

In [135]:
raw_documents = loader.load()

In [136]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(raw_documents)

In [137]:
embeddings = HuggingFaceEmbeddings() # Removed the groq_api_key argument

/tmp/ipython-input-4100843026.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings() # Removed the groq_api_key argument


In [147]:
# %pip install chromadb

In [138]:
from langchain.vectorstores import Chroma

# Create a vector store from the documents and embeddings
vectorstore = Chroma.from_documents(documents, embeddings)

In [141]:
# Expose the vector store as a retriever
retriever = vectorstore.as_retriever()

In [142]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)

In [143]:
# Set up the RetrievalQA chain with the Groq model
llm = ChatGroq(groq_api_key=groq_api_key,         model_name="moonshotai/kimi-k2-instruct",
temperature=0)

qa = RetrievalQA.from_chain_type(
  llm,
  retriever=retriever,
  memory = memory
)

In [144]:
query = "What is given assignment in Spring 2024 Assignments on Stanford CS class CS231n: Deep Learning for Computer Vision ?"

In [145]:
result = qa({"query": query})

In [146]:
result["result"]

'The Spring 2024 assignments for Stanford CS231n: Deep Learning for Computer Vision are:\n\n- **Assignment #1**: Image Classification, kNN, SVM, Softmax, Fully Connected Neural Network  \n- **Assignment #2**: Fully Connected and Convolutional Nets, Batch Normalization, Dropout, PyTorch & Network Visualization  \n- **Assignment #3**: Network Visualization, Image Captioning with RNNs and Transformers, Generative Adversarial Networks, Self-Supervised Contrastive Learning'

In [152]:
question1 = "According to the website, who are the instructors for the CS231n course?"

In [153]:
result = qa({"query": question1})

In [151]:
result['result']

"I carefully reviewed the provided context, which includes course notes, assignments, and module descriptions for CS231n. However, I did not find any mention of the instructors' names in the text provided. The context focuses on the course content and structure rather than listing the teaching staff.\n\n**Answer:** The instructors for the CS231n course are not mentioned in the provided context."

In [162]:
question2 ="Can you tell me?, what are topic given in Module 2: Convolutional Neural Networks?"

In [163]:
result = qa({'query':question2})

In [164]:
result['result']

'In Module 2: Convolutional Neural Networks, the following topics are covered:\n\n1. **Convolutional Neural Networks: Architectures, Convolution / Pooling Layers**  \n   - layers, spatial arrangement, layer patterns, layer sizing patterns, AlexNet/ZFNet/VGGNet case studies, computational considerations\n\n2. **Understanding and Visualizing Convolutional Neural Networks**  \n   - t-SNE embeddings, deconvnets, data gradients, fooling ConvNets, human comparisons\n\n3. **Transfer Learning and Fine-tuning Convolutional Neural Networks**'